In [ ]:
import importlib
import layers.tied
importlib.reload(layers.tied)
import tensorflow as tf
import datetime
#tf.enable_eager_execution()
print(tf.__version__)
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
from layers.tied import TiedDenseLayer
import tensorflow as tf
from tensorflow.python.keras.models import load_model
m = load_model("/Users/talfranji/Downloads/model2way.h5")

In [ ]:
#layers.tied._test_TiedDenseLayer()
#layers.tied._test_TiedDropoutLayer()
#layers.tied._test_LocallyDenseLayer()
layers.tied._test_all()

In [27]:
def eval_tensor(t):
    return tf.Session().run(t)
def tensor2const(t):
    return tf.constant(eval_tensor(t))

Rx = tensor2const(tf.random.uniform([1,100]))
Ry = tensor2const(tf.random.uniform([1,50]))

model = tf.keras.models.Sequential([
#tf.keras.layers.Flatten(),
tf.keras.layers.Dense(50, activation=tf.nn.softmax),
tf.keras.layers.Reshape([50,1]),
])
log_dir='/Users/talfranji/tmp/log/fit'
tb = TensorBoard(log_dir=log_dir)
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(Rx, Ry, epochs=5, steps_per_epoch=1, callbacks=[tb])

Epoch 1/5
1/1 [==============================] - 0s 225ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - acc: 0.0000e+00


In [ ]:

#eval_tensor(tf.concat([tf.constant([1,2,3]), tf.constant([4,5,6])], axis=0))
#eval_tensor(tf.constant([4,5,6]))

In [ ]:
Rx = layers.tied.tensor2const(tf.ones([1, 10, 10]))
Ry = layers.tied.tensor2const(tf.ones([1, 10, 10]))

rate = 0.5
inputs = tf.keras.Input(shape=(10, 10))
#TD1 = tf.layers.Dropout(rate=rate)
TD1 = layers.tied.TiedDropoutLayer()
TD1.build(inputs.shape)
TD2 = layers.tied.TiedDropoutLayer(tied_mask_variable=TD1.get_mask_varible())
outputs = TD2(TD1(inputs))

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.train.MomentumOptimizer(use_nesterov=True, learning_rate=0.001, momentum=0.8)
model.compile(optimizer, loss='mean_squared_error')

log_dir = "/tmp/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
model.fit(Rx, Ry, epochs=100, steps_per_epoch=1, callbacks=[tensorboard_callback])


In [1]:
# RESTART - start here after kernel restart with same machine   ##############################
!rm -fR /Users/talfranji/tmp/log/fit
import sys
import importlib
#path = "/content/2WayNet-TF"
#if not path in sys.path:
#  sys.path.append(path)
import run_model
importlib.invalidate_caches() # allow dirsync to change files
importlib.reload(run_model)

<module 'run_model' from '/Users/talfranji/Dropbox/C/lior_wolf/2WayNet-TF/run_model.py'>

In [2]:
m = run_model.run_model("/Users/talfranji/Dropbox/C/lior_wolf/2WayNet-TF/MNIST.ini")


data_parameters[name] = MNIST
data_parameters[path] = /Users/talfranji/data/MNIST/
data_parameters[normalize] = 1
data_parameters[scale] = 0
data_parameters[scale_samples] = 0
data_parameters[whiten] = 0
data_parameters[pca] = 0 0
data_parameters[full] = 0
data_parameters[subsample_test] = 0
Layer X->Y build <class 'layers.tied.TiedDenseLayer'>(units=392,input_shape=(None, 392))
Layer Y->X build <class 'layers.tied.TiedDenseLayer'>(kwargs={'units': 392, 'tied_layer': <layers.tied.TiedDenseLayer object at 0xb31f93eb8>})
Noise X->Y build <class 'layers.tied.TiedDropoutLayer'>(input_shape=(None, 392))
Layer X->Y build <class 'layers.tied.TiedDenseLayer'>(units=50,input_shape=(None, 392))
Layer Y->X build <class 'layers.tied.TiedDenseLayer'>(kwargs={'units': 392, 'tied_layer': <layers.tied.TiedDenseLayer object at 0x12eade828>})
Noise X->Y build <class 'layers.tied.TiedDropoutLayer'>(input_shape=(None, 50))
Layer X->Y build <class 'layers.tied.TiedDenseLayer'>(units=392,input_shape=(None, 

1000/1000 [==============================] - 8s 8ms/step - loss: 141.2700 - tied_dense_layer_6_loss: 6.7070 - tied_dense_layer_1_loss: 6.7070 - tied_dense_layer_6_dummy_metic_for_images: 128.1280 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_dense_layer_1_dummy_metic_for_images: 128.1280 - tied_dense_layer_1_metric_learning_rate: 1.0000e-04
Epoch 20/80
1000/1000 [==============================] - 8s 8ms/step - loss: 140.5066 - tied_dense_layer_6_loss: 6.4253 - tied_dense_layer_1_loss: 6.4253 - tied_dense_layer_6_dummy_metic_for_images: 119.2080 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_dense_layer_1_dummy_metic_for_images: 119.2080 - tied_dense_layer_1_metric_learning_rate: 1.0000e-04
Epoch 21/80
1000/1000 [==============================] - 8s 8ms/step - loss: 140.6130 - tied_dense_layer_6_loss: 6.5782 - tied_dense_layer_1_loss: 6.5782 - tied_dense_layer_6_dummy_metic_for_images: 171.2240 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_den

1000/1000 [==============================] - 8s 8ms/step - loss: 135.6722 - tied_dense_layer_6_loss: 6.0425 - tied_dense_layer_1_loss: 6.0425 - tied_dense_layer_6_dummy_metic_for_images: 136.6000 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_dense_layer_1_dummy_metic_for_images: 136.6000 - tied_dense_layer_1_metric_learning_rate: 1.0000e-04
Epoch 42/80
1000/1000 [==============================] - 8s 8ms/step - loss: 135.5413 - tied_dense_layer_6_loss: 6.0709 - tied_dense_layer_1_loss: 6.0709 - tied_dense_layer_6_dummy_metic_for_images: 111.2640 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_dense_layer_1_dummy_metic_for_images: 111.2640 - tied_dense_layer_1_metric_learning_rate: 1.0000e-04
Epoch 43/80
1000/1000 [==============================] - 8s 8ms/step - loss: 135.3966 - tied_dense_layer_6_loss: 6.0921 - tied_dense_layer_1_loss: 6.0921 - tied_dense_layer_6_dummy_metic_for_images: 92.8720 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_dens

1000/1000 [==============================] - 8s 8ms/step - loss: 131.0342 - tied_dense_layer_6_loss: 5.7285 - tied_dense_layer_1_loss: 5.7285 - tied_dense_layer_6_dummy_metic_for_images: 176.5280 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_dense_layer_1_dummy_metic_for_images: 176.5280 - tied_dense_layer_1_metric_learning_rate: 1.0000e-04
Epoch 64/80
1000/1000 [==============================] - 10s 10ms/step - loss: 130.6476 - tied_dense_layer_6_loss: 5.6235 - tied_dense_layer_1_loss: 5.6235 - tied_dense_layer_6_dummy_metic_for_images: 111.3120 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_dense_layer_1_dummy_metic_for_images: 111.3120 - tied_dense_layer_1_metric_learning_rate: 1.0000e-04
Epoch 65/80
1000/1000 [==============================] - 9s 9ms/step - loss: 130.7861 - tied_dense_layer_6_loss: 5.7808 - tied_dense_layer_1_loss: 5.7808 - tied_dense_layer_6_dummy_metic_for_images: 165.0400 - tied_dense_layer_6_metric_learning_rate: 1.0000e-04 - tied_d